<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/post_ollama_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import os
import time

In [18]:
#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [19]:
#un thread demone avvia il server locale di ollama
import subprocess
import threading
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
t.start()

In [20]:
#un altro thread demone avvia llama3
!ollama pull llama3
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3"]),daemon=True)
t2.start()

2024/06/30 16:32:29 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-06-30T16:32:29.596Z level=INFO source=images.go:730 msg="total blobs: 5"
time=2024-06-30T16:32:29.596Z level=INFO source=images.go:737 msg="total unused blobs removed: 0"
time=2024-0

[GIN] 2024/06/30 - 16:32:34 | 200 |      71.386µs |       127.0.0.1 | HEAD     "/"


time=2024-06-30T16:32:34.246Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu_avx2 cuda_v11 rocm_v60101 cpu cpu_avx]"
time=2024-06-30T16:32:34.365Z level=INFO source=types.go:98 msg="inference compute" id=GPU-b8c1fbd7-a663-db05-3763-9ec20ff0e227 library=cuda compute=6.0 driver=12.2 name="Tesla P100-PCIE-16GB" total="15.9 GiB" available="15.6 GiB"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ [GIN] 2024/06/30 - 16:32:35 | 200 |  717.975196ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [ ]:
moderator_instructions = [
    "You are a moderator. Given the following rules, please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n",
    "You are a owner of the Social. Given the following rules,please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n",
    "You are a dictator moderator. Given the following rules, please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n"
]

num_profili = 4


In [21]:
def ask_to_llama(instruction,rules, prompt):    
    full_context = instruction
    for i, rule in enumerate(rules, start=1):
        full_context += f"{i}. {rule['text']}\n"
    
    full_prompt = full_context + prompt
    
    response = requests.post('http://localhost:11434/api/generate', 
                             data=json.dumps({'model': 'llama3', 'prompt': full_prompt, 'stream': False}), 
                             headers={'Content-Type': 'application/json'})
    
    return response.json()['response']

In [26]:
def process_json_files(instances):
    
    start_time = time.time()
    instanze = os.listdir(instances)
    
    
    for filename in instanze:
        if filename.endswith('.json'):
            filepath = os.path.join(instances, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                rules = data['rules']
                records = data['records']
                records_en=[]
                
                for record in records:
                    lan = record['language']
                    if lan=="en":
                        records_en.append(record)
                        
                if len(records_en)==0:
                    print(f"Per l'istanza {data['name']} non ci sono post in inglese")
                    filt_recorods = []
                    
                if len(records_en)>200: #se len>200 prendiamo solo testa e coda
                    records_ordinati = sorted(records_en, key=lambda x: x['favourites'])
                    primi_100 = records_ordinati[:100]
                    ultimi_100 = records_ordinati[-100:]
                    filt_records = primi_100 + ultimi_100
                    
                else: #altrimenti li prendiamo tutti
                    filt_records = records_en
                
                #i=0
                for record in filt_records:
                    record_text = record['text']
                    #i+=1
                    for instruction in moderator_instructions:
                        response = ask_to_llama(instruction,rules,record_text)
                        print(f"Response for record ID {record['id']} with instruction '{instruction.split()[3]}': {response}")
                        '''if (i%100 == 0):
                            tempo = time.time() - start_time
                            print(f"PROCESSATE {i} in tempo: ",tempo)'''
                        #break #così vediamo cosa fa con le altre instanze
                    #break 
        #break
    end_time = time.time() 
    duration = end_time - start_time
    print("DURATA: ", duration)

In [30]:
instances = '/kaggle/input/instance-json/results'
process_json_files(instances)
#len(os.listdir(instances))//num_profili


197

In [ ]:
pip install bertopic